In [130]:
from pybtex.database.input import bibtex
import os

import pandas as pd

In [219]:
bibfile_list = ['calhoun']
parser = bibtex.Parser()

for bibfile in bibfile_list:
    bibfilename = bibfile + '.bib'
    bibdata = parser.parse_file(bibfilename)

    # Convert bibtex entries to list of dictionaries
    dict_list = []
    for bib_id in bibdata.entries:
        b = bibdata.entries[bib_id].fields
        
        # Replace {} in title
        t = b['title']
        t = t.replace('{','')
        t = t.replace('}','')
        t = t.replace('\sc','')
        b['title'] = t
                
        b['cite-key'] = bib_id
        b['type'] = bibdata.entries[bib_id].type
        dict_list.append(dict(b.items()))
    
df = pd.DataFrame(dict_list)

from numpy import nan
cols = df.columns
if 'image' not in cols:
    df['image'] = nan
    
print(df.columns)
cols = ['type','cite-key','title','year','local-url']
# cols = ['type','journal','booktitle','howpublished','series','institution']
display(df[cols])

Index(['date-added', 'date-modified', 'howpublished', 'institution',
       'keywords', 'local-url', 'title', 'year', 'abstract', 'bdsk-file-1',
       'cite-key', 'type', 'url', 'annote', 'bdsk-url-1', 'doi', 'journal',
       'number', 'pages', 'volume', 'booktitle', 'eprint', 'publisher',
       'series', 'bdsk-url-2', 'caption', 'image', 'isbn', 'address', 'month',
       'organization', 'issn', 'key'],
      dtype='object')


,type,cite-key,title,year,local-url
0,techreport,ai-ca-ha-pe-se-wr:2018,Massively Parallel Solvers for Computational F...,2018,ai-ca-ha-pe-se-wr-2018.pdf
1,webpage,amrresources,AMR Resources,2017,NaN
2,article,be-ca-he-le:2009,Logically rectangular finite volume methods wi...,2009,be-ca-he-le-2000.pdf
3,inproceedings,bu-ca-ma-te:2014,ForestClaw: Hybrid forest-of-octrees AMR for h...,2014,bu-ca-ma-te-2014.pdf
4,article,ca-bu:2017,ForestClaw : A parallel algorithm for patch-ba...,2017,ca-bu-2017.pdf
5,article,ca-ch-he:2022,The Cartesian Grid Active Flux Method with Ada...,2022,ca-ch-he-2022.pdf
6,techreport,ca-co:2010,Thermodynamics and transport of homogenous air...,2010,NaN
7,article,ca-he-le:2008,Logically rectangular grids and finite volume ...,2008,ca-he-le-2008.pdf
8,inproceedings,ca-he-le:2008b,A Finite Volume Grid for Solving Hyperbolic Pr...,2008,ca-he-le-2008b.pdf
9,article,ca-he:2009,A Finite Volume Method for Solving Parabolic E...,2009,ca-he-2009.pdf


In [220]:
bib_type_list = df.type.unique()
print(bib_type_list)

['techreport' 'webpage' 'article' 'inproceedings']


In [221]:
# Create a map to get info that can be used in a citation.

bib_type_map = {'article' : 'journal', \
                'inproceedings' : 'booktitle', \
                'webpage' : 'url', \
                'techreport' : 'institution', \
                'incollection' : 'booktitle', \
                'conference' : 'booktitle',}

In [222]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [223]:
# df_journals = df[df['journal'].isnull() == False]
def single_quote(s):
    return "'" + s + "'"

def double_quote(s):
    return "\"" + s + "\""

def endl():
    return "\n"

def br():
    return "<br/>"

refereed = ['article', 'inproceedings']

for row, item in df.iterrows():    
    bib_type = item['type']
    bib_id = item['cite-key']
    citekey = bib_id.replace(':','-')
    md_filename = citekey + ".md"
    html_filename = citekey + ".html"
    pdf_filename = citekey + ".pdf"
    year = item['year']
    venue = bib_type
    venue_pretext = item[bib_type_map[bib_type]]
    
    ## YAML variables
    
    md = "---" + endl()
    # ------
    # title: 
    # ------
    md += "title: " + single_quote(item['title']) + endl()
    
    # -----------
    # collection: 
    # -----------
    md += "collection: publications" + endl()

    
    # -----------
    # publish_type:
    # -----------
    if bib_type in refereed:
        publish_type = 'journal-article'
    else:
        publish_type = 'other'            
    md += "publish_type: " + publish_type + endl()
    
    # ----------
    # permalink: 
    # ----------
    # md += "permalink: /publication/" + html_filename + endl()
    md += 'permalink: /publication/' + publish_type + '/' + html_filename + endl()
    
    # ---------
    # abstract:
    # ---------
    if not pd.isna(item['abstract']):
        md += "abstract: " + single_quote(html_escape(item['abstract'])) + endl()

    # -----
    # date: 
    # -----
    md += "date: " + '01/01/' + item['year'] + endl()
    
    # ------
    # venue: 
    # ------
    md += "venue: " + single_quote(venue_pretext) + endl()
    
    # ---------
    # paperurl: 
    # ---------
    
    if not pd.isna(item['url']):
        url = item['url']
        md += "paperurl: " + single_quote(html_escape(url)) + endl()
    elif not pd.isna(item['doi']):
        doi = item['doi']
        md += "paperurl: " + single_quote(html_escape(doi)) + endl()
    
    # ---------
    # citation:
    # ---------
    citation = ""

    # citation authors - todo - add highlighting for primary author?
    for author in bibdata.entries[bib_id].persons["author"]:
        citation += author.first_names[0] + " " + author.last_names[0]+", "

    # citation title
    citation += double_quote(item['title']) + ","

    # Name of journal,book, etc
    citation += " " + "<i>" + venue_pretext + "</i>"
        
    citation += ", " + year + "."
    
    # This will use 'paperurl'
    md += "citation: " + single_quote(citation) + endl()

    # ----------------------------------------
    # images
    # ----------------------------------------
    
    if not pd.isna(item['image']):
        md += "images: " + single_quote(item['image']) + endl()
        if not pd.isna(item['caption']):
            md += "caption: " + single_quote(item['caption']) + endl()
    
    # ----------------------------------------
    # Download
    # ----------------------------------------
    
    if not pd.isna(item['local-url']):
        md += 'download: ' + item['local-url']+ endl()
        
        
    md += "---" + endl()
    
    
    print("Processed file : {:s}".format(citekey))
    print(citation)
    print(bib_type)
    print(venue_pretext)
    print(publish_type)
    print("")

    
    # ----------------------------------------
    # Markdown description for individual page
    # ----------------------------------------
    
    # Put miscellaneous content here
    
    # md += ...
    
    # ----------------------------------------
    # Write out file name
    # ----------------------------------------

    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)    

Processed file : ai-ca-ha-pe-se-wr-2018
Scott Aiton, Donna Calhoun, Jaber Hasbestan, Brenton Peck, Inanc Senocak, Grady Wright, "Massively Parallel Solvers for Computational Fluid Dynamics on Multi-block Cartesian Grids", <i>BSU Undergraduate Showcase (poster presentation)</i>, 2018.
techreport
BSU Undergraduate Showcase (poster presentation)
other

Processed file : amrresources
Donna Calhoun, "AMR Resources", <i>http://math.boisestate.edu/~calhoun/www_personal/research/amr_software/index.html</i>, 2017.
webpage
http://math.boisestate.edu/~calhoun/www_personal/research/amr_software/index.html
other

Processed file : be-ca-he-le-2009
Marsha Berger, Donna Calhoun, Christiane Helzel, Randall LeVeque, "Logically rectangular finite volume methods with adaptive refinement on the sphere", <i>Phil. Trans. R. Soc. A</i>, 2009.
article
Phil. Trans. R. Soc. A
journal-article

Processed file : bu-ca-ma-te-2014
Carsten Burstedde, Donna Calhoun, Kyle Mandli, Andy Terrel, "ForestClaw: Hybrid forest-o